In [84]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pandas as pd

In [85]:
load_dotenv()

True

In [86]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [87]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [88]:
embedding = OpenAIEmbeddings()

In [89]:
ASTRA_DB_API_ENDPOINT='https://8a9d3acd-528e-4be1-bb1e-1b17b4cf9399-us-east-2.apps.astra.datastax.com'

In [90]:
ASTRA_DB_APPLICATION_TOKEN='AstraCS:ZQHAyMdrvWnnhgPkZMSQWsFf:7732f6f0e7f615cfc4779803761731f608d8f99cddd189a72cecbc58852fdadf'

In [91]:
ASTRA_DB_KEYSPACE= "default_keyspace"

In [92]:
collection_name= "financebot"

In [93]:
vstore = AstraDBVectorStore(
    embedding= embedding,
    collection_name= collection_name,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE
    )

In [94]:
from langchain_community.document_loaders import PyPDFLoader

In [95]:
loader = PyPDFLoader("C:\\Users\\svani\\OneDrive\\Documents\\Inueron-7-projects\\Trading-Chatbot\\data\\finance_data.pdf")

In [96]:
pages = loader.load()

In [97]:
len(pages)

108

In [98]:
raw_text = ''
for i, doc in enumerate(pages):
    text = doc.page_content
    if text:
        raw_text += text

In [99]:
print(raw_text)

PDF’s filings with the Securities and Exchange Commission, including any changes in the marketplace for semiconductor analytics products, including the introduction of 
products or services competitive with those of PDF. The forward‐looking statements made in this Report are made as of the date hereof, and PDF does not assume any 
obligation to update such statements nor the reasons why actual results could differ materially from those projected in such statements.
© 2022 PDF Solutions, Inc. All rights reserved. Trademarks used herein are the property of PDF Solutions, Inc. or its affiliates. CORPOR ATE INFORM ATION
MANAGEMENT TEAM 
John K. Kibarian, Ph.D.  
Chief Executive Office r,   
President and Co‐Founder 
Adnan Raza  
Executive Vice President, Finance  
and Chief Financial Officer  
Kimon Michaels, Ph.D.  
Executive Vice President, 
Products  and Solutions and Co‐
Founder 
Andrzej Strojwas, Ph.D.  
Chief Technology OfficerBOARD OF DIRECTORS 
Joseph R. Bronson  
Lead Independent 

In [100]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [101]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
)

In [102]:
texts = text_splitter.split_text(raw_text)

In [103]:
len(texts)

941

In [104]:
from langchain_core.documents import Document

In [105]:
docs = []
for i in range(len(texts)):
    doc = Document(page_content=texts[i])
    docs.append(doc)

In [106]:
vstore.add_documents(docs)

['97e18708060a4c6fb0b24a1dfdb2468a',
 'c2b6ca39181c4f74812a77a8e6b59331',
 'ff00667a84da4c4597f07d44e21e4d91',
 '0dcfb771946c46da95acfcaa23326c30',
 'a1887db0cc5e4faebdf31227f65535c8',
 '3e4990d49e214ecf9da27e2826208e8d',
 '9e9c47b186864ad29116d22e994a6030',
 'af01022dc2d74a8b96246eee34889695',
 '43772688ef904e44b89a513f38c6585e',
 '857527127e934fe78fceae560c3ed02f',
 '0dee9ab1ecf2412e8f3539fd3fbfcb10',
 '296355582c3d44a48fb322dc29362b66',
 '70b3c18aa8754bcb9b02007059cfc1ce',
 'bc3685fa190047708fe65d434faae6aa',
 '14af9d5186324c04b90e374a8166288b',
 'aa97da1a3aaa4f8c85bc73c61d25cfde',
 '54dc1b65c00c482d9235bcc87a0be774',
 '419b7751ea5f4258a351c813a12f0872',
 '847abbe012c74512be4102b4ef96dd3c',
 '89d5dbc350364509be81ce4f11b69cf7',
 'b9b996e5bcca482a8f22f00389925fd8',
 '6d9c57df4a1045ca85b4e7c9feda43fb',
 'fe1a936de45741d6a6e0128fcc448f86',
 'b05dd82a899045109b5803b2fa538b3a',
 '0b0ad3781f7b4dc9a36c6a19a16d3ca7',
 '861addff759c4367915bc79e0da8e973',
 'd2075438670c4108b861c6a2a0149627',
 

In [107]:
retriever = vstore.as_retriever(search_kwargs={"k": 3})

In [108]:
retriever.get_relevant_documents("what is Market For Registrant’s Common Equity?")

[Document(page_content='Item 4. Mine Safety Disclosures \nNot applicable. \n  Table of Contents \n38  \nPART II \nItem 5. Market For Registrant’s Common Equity, and Relate d Stockholder Matters and Issuer Purchases of Equity \nSecurities \nOur common stock trades on the Na sdaq Global Market under the s ymbol “PDFS.” As of February 23, 2024, we had \napproximately 24 stockholders of record. The number of stockhol ders of record does not include individuals whose stock'),
 Document(page_content='Item 4. Mine Safety Disclosures \nNot applicable. \n  Table of Contents \n38  \nPART II \nItem 5. Market For Registrant’s Common Equity, and Relate d Stockholder Matters and Issuer Purchases of Equity \nSecurities \nOur common stock trades on the Na sdaq Global Market under the s ymbol “PDFS.” As of February 23, 2024, we had \napproximately 24 stockholders of record. The number of stockhol ders of record does not include individuals whose stock'),
 Document(page_content='Item 4. Mine Safety Disc

In [109]:
PRODUCT_BOT_TEMPLATE = """
    Your finance bot is an expert in finance related advice.
    Ensure your answers are relevant to the query context and refrain from straying off-topic.
    Your responses should be concise and informative.

    CONTEXT:
    {context}

    QUESTION: {question}

    YOUR ANSWER:
    
    """

In [110]:
from langchain_core.prompts import ChatPromptTemplate

In [111]:
prompt = ChatPromptTemplate.from_template(PRODUCT_BOT_TEMPLATE)

In [112]:
from langchain_openai import ChatOpenAI

In [113]:
llm = ChatOpenAI()

In [114]:
from langchain_core.runnables import RunnablePassthrough

In [115]:
from langchain_core.output_parsers import StrOutputParser

In [116]:
chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

In [117]:
chain.invoke("what is Market For Registrant’s Common Equity?")

'The Market for Registrant\'s Common Equity refers to the trading platform or exchange where a company\'s common stock is bought and sold by investors. In this case, the company\'s common stock trades on the Nasdaq Global Market under the symbol "PDFS."'